#  Introduction

The term "melting pot" was coined to describe New York's densely populated immigrant neighborhoods. The diversity of culture and food products is great. There are many restaurants in New York City, each belonging to a different category such as Chinese, Indian, French, etc. Therefore, in this project, we will make a list of the main areas of New York City where there are good French restaurants and visualize them. Data

For this project, we need the following data:

 New York City data that lists boroughs, neighborhoods, and their latitude and longitude. 
    Data source: https://cocl.us/new_york_dataset
    
 French restaurants in every neighborhood in New York City. Data source : API Fousquare with the French restaurant filter




### Approach:

Describe the dataset on newyork.
Filter all places that are French restaurants .
Find info for every French restaurant in newyork using the FourSquare API.
Using the info for each restaurant, we will sort this data.
Then we will view the ranking

### Librairies:

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import os
import folium 
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
%matplotlib inline



## Creation des datasets 

### NewYork data set

In [2]:
def get_new_york_data():
    url='https://cocl.us/new_york_dataset'
    resp=requests.get(url).json()
    features=resp['features']
    column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 
    df_new_york = pd.DataFrame(columns=column_names)
    
    for data in features:
        borough = data['properties']['borough'] 
        neighborhood_name = data['properties']['name']
        
        neighborhood_latlon = data['geometry']['coordinates']
        neighborhood_lat = neighborhood_latlon[1]
        neighborhood_lon = neighborhood_latlon[0]
    
        df_new_york = df_new_york.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
    
    return df_new_york


In [3]:
df_new_york=get_new_york_data()

In [4]:
df_new_york.shape

(306, 4)

In [5]:
df_new_york.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [6]:
print("There is "+ str(df_new_york['Borough'].unique().shape[0]) +" Borough in NewYork")
print("There is "+ str(df_new_york['Neighborhood'].unique().shape[0]) +" Neighborhood in NewYork")


There is 5 Borough in NewYork
There is 302 Neighborhood in NewYork


In [7]:
print("There is "+ str(df_new_york["Borough"][(df_new_york['Borough'] == "Bronx") ].count()) +" Neighborhood in Borough Bronx")
print("There is "+ str(df_new_york["Borough"][(df_new_york['Borough'] == "Manhattan") ].count()) +" Neighborhood in Borough Manhattan")
print("There is "+ str(df_new_york["Borough"][(df_new_york['Borough'] == "Brooklyn") ].count()) +" Neighborhood in Borough Brooklyn")
print("There is "+ str(df_new_york["Borough"][(df_new_york['Borough'] == "Queens") ].count()) +" Neighborhood in Borough Queens")
print("There is "+ str(df_new_york["Borough"][(df_new_york['Borough'] == "Staten Island") ].count()) +" Neighborhood in Staten Island")


There is 52 Neighborhood in Borough Bronx
There is 40 Neighborhood in Borough Manhattan
There is 70 Neighborhood in Borough Brooklyn
There is 81 Neighborhood in Borough Queens
There is 63 Neighborhood in Staten Island


### List of French Restaurant in NewYork 

In [8]:
def get_venues(lat,lng):
    radius=1000
    LIMIT=100
    CLIENT_ID = ""
    CLIENT_SECRET = ""
    VERSION = '20201105' # Foursquar
    
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
    
    results = requests.get(url).json()
    venue_data=results["response"]['groups'][0]['items']
    venue_details=[]
    for row in venue_data:
        try:
            venue_id=row['venue']['id']
            venue_name=row['venue']['name']
            venue_category=row['venue']['categories'][0]['name']
            venue_details.append([venue_id,venue_name,venue_category])
        except KeyError:
            pass
        
    column_names=['ID','Name','Category']
    df = pd.DataFrame(venue_details,columns=column_names)
    return df

In [12]:
column_names=['Borough', 'Neighborhood', 'ID','Name']
df_french_restaurant_new_york=pd.DataFrame(columns=column_names)
count=1
for row in df_new_york.values.tolist():
    Borough, Neighborhood, Latitude, Longitude=row
    venues = get_venues(Latitude,Longitude)
    french_restaurant_ny=venues[venues['Category']=='French Restaurant']   
    for resturant_detail in french_restaurant_ny.values.tolist():
        id, name , category=resturant_detail
        df_french_restaurant_new_york = df_french_restaurant_new_york.append({'Borough': Borough,
                                                'Neighborhood': Neighborhood, 
                                                'ID': id,
                                                'Name' : name
                                               }, ignore_index=True)

In [13]:
df_french_restaurant_new_york.head()

,Borough,Neighborhood,ID,Name
0,Bronx,City Island,4d7aa83ee540f04d66ba16dc,Bistro SK
1,Brooklyn,Greenpoint,58fd34584382ab70ad90083e,Chez Ma Tante
2,Brooklyn,Windsor Terrace,4e7d174cb6340a4da75a2cca,Le Paddock
3,Brooklyn,Bedford Stuyvesant,5003007de4b06d50188aa593,Le Paris Dakar
4,Brooklyn,Brooklyn Heights,4f7f8b86e4b088077df30175,Chez Moi


In [14]:
df_french_restaurant_new_york.shape

(62, 4)

In [15]:
print("There is "+ str(df_french_restaurant_new_york.shape[0]) +" french restaurant in NewYork")
print("There is "+ str(df_french_restaurant_new_york['Neighborhood'].unique().shape[0]) +" Neighborhood in NewYork who have french restaurant")


There is 62 french restaurant in NewYork
There is 43 Neighborhood in NewYork who have french restaurant


In [16]:
print("There is "+ str(df_french_restaurant_new_york["Borough"][(df_french_restaurant_new_york['Borough'] == "Bronx") ].count()) +" Neighborhood in Borough Bronx")
print("There is "+ str(df_french_restaurant_new_york["Borough"][(df_french_restaurant_new_york['Borough'] == "Manhattan") ].count()) +" Neighborhood in Borough Manhattan")
print("There is "+ str(df_french_restaurant_new_york["Borough"][(df_french_restaurant_new_york['Borough'] == "Brooklyn") ].count()) +" Neighborhood in Borough Brooklyn")
print("There is "+ str(df_french_restaurant_new_york["Borough"][(df_french_restaurant_new_york['Borough'] == "Queens") ].count()) +" Neighborhood in Borough Queens")
print("There is "+ str(df_french_restaurant_new_york["Borough"][(df_french_restaurant_new_york['Borough'] == "Staten Island") ].count()) +" Neighborhood in Staten Island")


There is 1 Neighborhood in Borough Bronx
There is 37 Neighborhood in Borough Manhattan
There is 16 Neighborhood in Borough Brooklyn
There is 6 Neighborhood in Borough Queens
There is 2 Neighborhood in Staten Island


We see that Manhattan has the largest number of French resturants


### Add informations of French Restaurant in NewYork for create Ranking List  

In [45]:
def get_venue_details(venue_id):
        
    CLIENT_ID = ""
    CLIENT_SECRET = ""  
    VERSION = '20201105'
    LIMIT=100
    
    url = 'https://api.foursquare.com/v2/venues/{}?&client_id={}&client_secret={}&v={}&limit={}'.format(
            venue_id,
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION,
            LIMIT)
    
    results = requests.get(url).json()
    venue_data=results['response']['venue']
    venue_details=[]
    try:
        venue_id=venue_data['id']
        venue_name=venue_data['name']
        venue_likes=venue_data['likes']['count']
        venue_rating=venue_data['rating']
        venue_tips=venue_data['tips']['count']
        venue_details.append([venue_id,venue_name,venue_likes,venue_rating,venue_tips])
    except KeyError:
        pass
        
    column_names=['ID','Name','Likes','Rating','Tips']
    df = pd.DataFrame(venue_details,columns=column_names)
    return df

In [46]:
column_names=['Borough', 'Neighborhood', 'ID','Name','Likes','Rating','Tips']
df_french_restaurant_stats_new_york=pd.DataFrame(columns=column_names)

for row in df_french_restaurant_new_york.values.tolist():
    Borough,Neighborhood,ID,Name=row
    try:
        venue_details=get_venue_details(ID)
        id,name,likes,rating,tips=venue_details.values.tolist()[0]
    except IndexError:
        id,name,likes,rating,tips=[0]*5
    df_french_restaurant_stats_new_york = df_french_restaurant_stats_new_york.append({'Borough': Borough,
                                                'Neighborhood': Neighborhood, 
                                                'ID': id,
                                                'Name' : name,
                                                'Likes' : likes,
                                                'Rating' : rating,
                                                'Tips' : tips
                                               }, ignore_index=True)
print("OK")



OK


In [47]:
df_french_restaurant_stats_new_york.head()

,Borough,Neighborhood,ID,Name,Likes,Rating,Tips
0,Bronx,City Island,4d7aa83ee540f04d66ba16dc,Bistro SK,13,7.6,7
1,Brooklyn,Greenpoint,58fd34584382ab70ad90083e,Chez Ma Tante,212,8.7,58
2,Brooklyn,Windsor Terrace,4e7d174cb6340a4da75a2cca,Le Paddock,135,8.8,56
3,Brooklyn,Bedford Stuyvesant,5003007de4b06d50188aa593,Le Paris Dakar,145,8.4,41
4,Brooklyn,Brooklyn Heights,4f7f8b86e4b088077df30175,Chez Moi,201,8.4,75


In [48]:
df_french_restaurant_stats_new_york.shape

(62, 7)

In [49]:
df_french_restaurant_stats_new_york.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62 entries, 0 to 61
Data columns (total 7 columns):
Borough         62 non-null object
Neighborhood    62 non-null object
ID              62 non-null object
Name            62 non-null object
Likes           62 non-null object
Rating          62 non-null float64
Tips            62 non-null object
dtypes: float64(1), object(6)
memory usage: 3.5+ KB


In [51]:
df_french_restaurant_stats_new_york['Likes']=df_french_restaurant_stats_new_york['Likes'].astype('float64')
df_french_restaurant_stats_new_york['Tips']=df_french_restaurant_stats_new_york['Tips'].astype('float64')

In [55]:
df_french_restaurant_stats_new_york.iloc[df_french_restaurant_stats_new_york['Likes'].idxmax()]

Borough                        Manhattan
Neighborhood                West Village
ID              4d0bf7e3f29c236ac675bfe7
Name                             Buvette
Likes                               1851
Rating                               9.2
Tips                                 519
Name: 42, dtype: object

In [56]:
df_french_restaurant_stats_new_york.iloc[df_french_restaurant_stats_new_york['Rating'].idxmax()]

Borough                        Manhattan
Neighborhood                 East Harlem
ID              553c1b00498e7eb81316bf1b
Name                       Mountain Bird
Likes                                 80
Rating                               9.3
Tips                                  18
Name: 22, dtype: object

In [57]:
df_french_restaurant_stats_new_york.iloc[df_french_restaurant_stats_new_york['Tips'].idxmax()]


Borough                        Manhattan
Neighborhood                West Village
ID              4d0bf7e3f29c236ac675bfe7
Name                             Buvette
Likes                               1851
Rating                               9.2
Tips                                 519
Name: 42, dtype: object

In [58]:
ny_neighborhood_stats=df_french_restaurant_stats_new_york.groupby('Neighborhood',as_index=False).mean()[['Neighborhood','Rating']]
ny_neighborhood_stats.columns=['Neighborhood','Average Rating']
ny_neighborhood_stats.sort_values(['Average Rating'],ascending=False).head(10)

,Neighborhood,Average Rating
14,East Harlem,9.30
40,West Village,9.20
29,Noho,9.10
19,Greenwich Village,8.90
33,Soho,8.90
9,Civic Center,8.90
23,Little Italy,8.90
7,Chinatown,8.90
25,Lower East Side,8.85
42,Yorkville,8.80


In [59]:
ny_borough_stats=df_french_restaurant_stats_new_york.groupby('Borough',as_index=False).mean()[['Borough','Rating']]
ny_borough_stats.columns=['Borough','Average Rating']
ny_borough_stats.sort_values(['Average Rating'],ascending=False).head()

,Borough,Average Rating
2,Manhattan,8.691892
1,Brooklyn,8.293750
3,Queens,8.083333
0,Bronx,7.600000
4,Staten Island,7.200000


In [60]:
ny_neighborhood_stats=ny_neighborhood_stats[ny_neighborhood_stats['Average Rating']>=9.0]

In [61]:
ny_neighborhood_stats=pd.merge(ny_neighborhood_stats,df_new_york, on='Neighborhood')

In [62]:
ny_neighborhood_stats=ny_neighborhood_stats[['Borough','Neighborhood','Latitude','Longitude','Average Rating']]

In [77]:
ny_map1 = folium.Map(location=[40.693943, -73.985880], default_zoom_start=80)

In [78]:
incidents = folium.map.FeatureGroup()
for lat, lng, in ny_neighborhood_stats[['Latitude','Longitude']].values:
    incidents.add_child(
        folium.CircleMarker(
            [lat, lng],
            radius=10,
            color='yellow',
            fill=True,
            fill_color='blue',
            fill_opacity=0.6
        )
    )
ny_map1.add_child(incidents)

# Conclusion

Manahtan and notably East Harlem, West Village, Noho are where you have to go to eat a good French restaurant. The queen is a good place to put a French restaurant.